## Context Encoder

Let's begin by importing tensorflow and the network


In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (14, 28)
import IPython
import os
import librosa
import numpy as np
import tensorflow as tf

from network.sequentialModel import SequentialModel
from network.stftGapContextEncoder import StftGapContextEncoder
from utils.strechableNumpyArray import StrechableNumpyArray


Now we initialize the context encoder network and select the step we want to use for the reconstruction.

In [ ]:
sr = 16000
side_length = 2048
gap_length = 1024
window_size = side_length*2+gap_length

batch_size = 256
fft_frame_length = 512
fft_frame_step = 128


In [ ]:
test_step = #complete me

In [ ]:
tf.reset_default_graph()

aTargetModel = SequentialModel(shapeOfInput=(batch_size, window_size), name="Target Model")

with tf.name_scope('Remove_unnecesary_sides_before_stft'):
    signal = aTargetModel.output()
    signal_without_unnecesary_sides = signal[:, 1664:3456]
    aTargetModel.setOutputTo(signal_without_unnecesary_sides)
aTargetModel.addSTFT(frame_length=fft_frame_length, frame_step=fft_frame_step)
aTargetModel.divideComplexOutputIntoRealAndImaginaryParts()  # (256, 11, 257, 2)

aModel = SequentialModel(shapeOfInput=(batch_size, window_size), name="context encoder")

with tf.name_scope('Remove_gap_before_stft'):
    signal = aModel.output()
    left_side = signal[:, :2048]
    right_side = signal[:, 2048+1024:]
    
    # This is strange. The window is 5K samples long, the hole 1024 and the 0 pading 384.
    # Unless signal in in spectrogram. In that case, the code is not very clear. Maybe consider adding comments.
    left_side_padded = tf.concat((left_side, tf.zeros((batch_size, 384))), axis=1)
    right_side_padded = tf.concat((tf.zeros((batch_size, 384)), right_side), axis=1)

    # If you pad them with 0, maybe you also stack them allong axis 2 (one after the other.)
    signal_without_gap = tf.stack((left_side_padded, right_side_padded), axis=1)  # (256, 2, 2432)
    aModel.setOutputTo(signal_without_gap)

aModel.addSTFT(frame_length=fft_frame_length, frame_step=fft_frame_step)  # (256, 2, 16, 257)
aModel.addReshape((batch_size, 32, 257))
aModel.divideComplexOutputIntoRealAndImaginaryParts()  # (256, 32, 257, 2)
aModel.addReshape((batch_size, 16, 257, 4))

with tf.variable_scope("Encoder"):
    filter_shapes = [(7, 89), (3, 17), (2, 11), (1, 9), (1, 5), (2, 5)]
    input_channels = [4, 32, 128, 512, 256, 160]
    output_channels = [32, 128, 512, 256, 160, 128]
    strides = [[1, 2, 2, 1], [1, 2, 3, 1], [1, 2, 3, 1], [1, 1, 2, 1], [1, 1, 1, 1], [1, 1, 1, 1]]
    names = ['First_Conv', 'Second_Conv', 'Third_Conv', 'Fourth_Conv', 'Fifth_Conv', 'Sixth_Conv']
    aModel.addSeveralConvLayers(filter_shapes=filter_shapes, input_channels=input_channels,
                                output_channels=output_channels, strides=strides, names=names)

aModel.addReshape((batch_size, 2048))
aModel.addFullyConnectedLayer(2048, 2048, 'Fully')
aModel.addRelu()
aModel.addBatchNormalization()
aModel.addReshape((batch_size, 8, 8, 32))

with tf.variable_scope("Decoder"):
    filter_shapes = [(8, 8), (5, 5), (3, 3)]
    input_channels = [32, 128, 512]
    output_channels = [128, 512, 257]
    strides = [[1, 2, 2, 1], [1, 2, 2, 1], [1, 1, 1, 1]]
    names = ['First_Deconv', 'Second_Deconv', 'Third_Deconv']
    aModel.addSeveralDeconvLayers(filter_shapes=filter_shapes, input_channels=input_channels,
                                  output_channels=output_channels, strides=strides, names=names)

    aModel.addReshape((batch_size, 8, 257, 128))
    aModel.addDeconvLayer(filter_shape=(5, 67), input_channels=128, output_channels=11, stride=(1, 2, 2, 1),
                          name='Fourth_deconv')
    aModel.addBatchNormalization()

    aModel.addReshape((batch_size, 11, 257, 32))

    aModel.addDeconvLayerWithoutNonLin(filter_shape=(11, 257), input_channels=32, output_channels=2,
                                       stride=(1, 1, 1, 1), name="Last_Deconv")

print(aModel.description())


In [ ]:
aContextEncoderNetwork = StftGapContextEncoder(model=aModel, batch_size=batch_size, target_model=aTargetModel, window_size=window_size,
                                               gap_length=gap_length, learning_rate=1e-4, name='nat_stft_gap_big_fma_2_')


In [ ]:
batch_count = 2
test_tfRecord = 'test-fma_w5120_g1024_h512_ex813108.tfrecords'
reconstructed_spec, original_spec = aContextEncoderNetwork.reconstruct(data_path=test_tfRecord, 
                                                                       model_num=test_step, max_steps=batch_count)

In [ ]:
print(reconstructed_spec.shape)

In [ ]:
tf.reset_default_graph()
rec_stft = reconstructed_spec[:,:,:,0] + 1.0j*reconstructed_spec[:,:,:,1]
ori_stft = original_spec[:,:,:,0] + 1.0j*original_spec[:,:,:,1]

shape = (batch_size, 11, 257)
stft_to_invert = tf.placeholder(tf.complex64, shape=shape, name='stft_to_invert')
tf_reconstructed_signals = tf.contrib.signal.inverse_stft(stfts=stft_to_invert, frame_length=fft_frame_length, frame_step=fft_frame_step)

reconstructed_signal = np.zeros([0,1792], dtype=float32)
original_signal = np.zeros([0,1792], dtype=float32)
with tf.Session() as sess:
    for i in range(batch_count):
        feed_dict = {stft_to_invert: rec_stft[i*batch_size:(i+1)*batch_size]}
        reconstructed_signal = np.append(reconstructed_signal, sess.run(tf_reconstructed_signals, feed_dict=feed_dict), axis=0)
        
        feed_dict = {stft_to_invert: ori_stft[i*batch_size:(i+1)*batch_size]}
        original_signal = np.append(original_signal, sess.run(tf_reconstructed_signals, feed_dict=feed_dict), axis=0)  

In [ ]:
reconstructed_gaps = reconstructed_signal[:, 384:-384]
original_gaps = original_signal[:, 384:-384]

In [ ]:
print(reconstructed_signal.shape)
print(original_signal.shape)
print(reconstructed_spec.shape)
print(original_spec.shape)
print(len(reconstructed_signal))


In [ ]:
def _pavlovs_SNR(y_orig, y_inp, onAxis=(1,)):
    norm_y_orig = _squaredEuclideanNorm(y_orig, onAxis)
    norm_y_orig_minus_y_inp = _squaredEuclideanNorm(y_orig - y_inp, onAxis)
    return 10 * np.log10(norm_y_orig / norm_y_orig_minus_y_inp)

def _squaredEuclideanNorm(vector, onAxis=(1,)):
    squared = np.square(vector)
    summed = np.sum(squared, axis=onAxis)
    return summed


In [ ]:
fake_a = reconstructed_gaps
gap = original_gaps

SNRs = _pavlovs_SNR(gap, fake_a)
print('SNRs shape:', SNRs.shape)
print('SNRs mean:', SNRs.mean())
print('SNRs std:', SNRs.std())
print('SNRs min:', SNRs.min())
print('SNRs percentiles 25, 50 & 75:', np.percentile(SNRs, [25, 50, 75]))
print('SNRs max:', SNRs.max())

spec_SNRs = _pavlovs_SNR(original_spec, 
                   reconstructed_spec, onAxis=(1, 2, 3))

print('SNRs on specs mean:', np.mean(spec_SNRs))

print('max diff for SNRs on specs:', np.max(np.abs(spec_SNRs-SNRs)))

In [ ]:
reconstructed_signal_to_evaluate = 20

In [ ]:
f, axarr = plt.subplots(1, 3, sharey='row', figsize=(18, 12))

difference = original_gaps[reconstructed_signal_to_evaluate]-reconstructed_gaps[reconstructed_signal_to_evaluate]

axarr[0].plot(original_gaps[reconstructed_signal_to_evaluate])
axarr[0].set_title('original gap', size=24)
axarr[1].plot(reconstructed_gaps[reconstructed_signal_to_evaluate])
axarr[1].set_title('reconstructed gap', size=24)
axarr[2].plot(difference)
axarr[2].set_title('difference', size=24)

print(np.sum(np.absolute(original_gaps[reconstructed_signal_to_evaluate])))
print(np.absolute(difference).sum())
print(np.linalg.norm(difference))

print('SNR:', _pavlovs_SNR(original_gaps[reconstructed_signal_to_evaluate], reconstructed_gaps[reconstructed_signal_to_evaluate], onAxis=0))



In [ ]:
f, axarr = plt.subplots(1, 4, sharey='row', figsize=(24, 12))

original_mag_spec = np.abs(original_spec[:, :, :, 0] + 1.0j*original_spec[:, :, :, 1])
rec_mag_spec = np.abs(reconstructed_spec[:, :, :, 0] + 1.0j*reconstructed_spec[:, :, :, 1])

rec_mag_to_plot = np.transpose(rec_mag_spec[reconstructed_signal_to_evaluate])
window_to_plot = np.transpose(original_mag_spec[reconstructed_signal_to_evaluate])

difference = window_to_plot-rec_mag_to_plot
print(window_to_plot.shape)
print(rec_mag_to_plot.shape)

print(np.zeros(reconstructed_spec[reconstructed_signal_to_evaluate].shape).shape)
z_min = np.min(window_to_plot)
z_max = np.max(window_to_plot)

axarr[0].pcolormesh(window_to_plot, vmin=z_min, vmax=z_max)
axarr[0].set_title('original', size=24)
axarr[1].pcolormesh(rec_mag_to_plot, vmin=z_min, vmax=z_max)
axarr[1].set_title('reconstructed', size=24)
axarr[2].pcolormesh(np.zeros(rec_mag_to_plot.shape), vmin=z_min, vmax=z_max)
axarr[2].set_title('zeroed', size=24)
axarr[3].pcolormesh(difference, vmin=z_min, vmax=z_max)
axarr[3].set_title('difference', size=24)

# real_for_snr = np.stack((np.real(original_stft[:, 13:24]), np.imag(original_stft[:, 13:24])),axis=-1)
print(_pavlovs_SNR(original_spec[reconstructed_signal_to_evaluate], reconstructed_spec[reconstructed_signal_to_evaluate], onAxis=(0,1,2)))

In [ ]:
stft_SNRs = _pavlovs_SNR(ori_stft, 
                   rec_stft, onAxis=(1, 2))

mag_SNRs = _pavlovs_SNR(np.abs(ori_stft), np.abs(rec_stft), onAxis=(1, 2))
phase_SNRs = _pavlovs_SNR(np.angle(ori_stft), np.angle(rec_stft), onAxis=(1, 2))

phasetimesmag_SNRs = _pavlovs_SNR(np.angle(ori_stft)*np.abs(ori_stft), np.abs(rec_stft)*np.angle(rec_stft), onAxis=(1, 2))

stft_channels_SNRs = _pavlovs_SNR(original_spec, 
                   reconstructed_spec, onAxis=(1, 2, 3))

print('SNRs on stft per channel mean:', np.mean(stft_channels_SNRs))
print('SNRs on stft mean:', np.mean(stft_SNRs))
print('SNRs on mag mean:', np.mean(mag_SNRs))
print('SNRs on phase mean:', np.mean(phase_SNRs))
print('SNRs on phase times mag mean:', np.mean(phasetimesmag_SNRs))


In [ ]:
sorted_SNR = sorted(SNRs)

import scipy.stats as stats
fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    